In [ ]:
import pandas as pd
import os
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import yfinance as yf

In [ ]:
symbol = "^VIX"
data_dir = '../data/%s' % symbol
data_file = '%s_history.csv' %symbol;
file_path = os.path.join(data_dir, data_file)

In [ ]:
data = pd.read_csv(file_path)

In [ ]:
data.head(5)

In [ ]:
price_pt_1 = 40
price_pt_2 = 20
fromDate = '2001-01-01'
toDate = '2021-03-01'
direction = 'UPTREND' if price_pt_1 < price_pt_2 else 'DOWNTREND'

In [ ]:
analysis_data_window = data[(data['Date'] >= fromDate) & (data['Date'] <= toDate)]

In [ ]:
analysis_data_window.head(5)

In [ ]:
analysis_data_window.tail(5)

In [ ]:
price_pt_1_all_occurences = analysis_data_window[analysis_data_window['High'] >= price_pt_1]

In [ ]:
price_pt_1_all_occurences.head(5)

In [ ]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
end_date_obj = datetime.datetime.strptime(toDate, '%Y-%m-%d')

pricePt1Date = []
pricePt2Date = []
calendarDays = []
businessDays = []
pricePt1 = []
pricePt2 = []

while True:
    price_pt_1_date = price_pt_1_all_occurences.iloc[0]['Date']
    filtered_data_set = analysis_data_window[analysis_data_window['Date'] >= price_pt_1_date]
    if direction == 'UPTREND':
        price_pt_2_date = filtered_data_set[filtered_data_set['Close'] >= price_pt_2].iloc[0]['Date']
    else:
        price_pt_2_date = filtered_data_set[filtered_data_set['Close'] <= price_pt_2].iloc[0]['Date']
    
    price_pt_1_date_obj = datetime.datetime.strptime(price_pt_1_date, '%Y-%m-%d')
    pricePt1DateObj.append(price_pt_1_date_obj);
    price_pt_2_date_obj = datetime.datetime.strptime(price_pt_2_date, '%Y-%m-%d')
    pricePt2DateObj.append(price_pt_2_date_obj);
    
    total_cal_days_for_price_change =  (price_pt_2_date_obj - price_pt_1_date_obj).days
    total_business_days_for_price_change = len(pd.date_range(start = price_pt_1_date, end = price_pt_2_date, freq = us_bd))
    
    pricePt1Date.append(price_pt_1_date)
    pricePt2Date.append(price_pt_2_date)
    calendarDays.append(total_cal_days_for_price_change)
    businessDays.append(total_business_days_for_price_change)
    pricePt1.append(price_pt_1)
    pricePt2.append(price_pt_2)
    
    price_pt_1_all_occurences = price_pt_1_all_occurences[price_pt_1_all_occurences['Date'] > price_pt_2_date]
    
    if len(price_pt_1_all_occurences.index) == 0 :
        break
       
        
out_data = {'Price_Pt_1': pricePt1,
        'Price_Pt_1_Date': pricePt1Date, 
        'Price_Pt_2': pricePt2,
        'Price_Pt_2_Date': pricePt2Date,
        'Calendar Days': calendarDays,
        'Business Days': businessDays}
output_df = pd.DataFrame(out_data, columns= ['Price_Pt_1', 'Price_Pt_1_Date', 'Price_Pt_2', 'Price_Pt_2_Date', 'Calendar Days', 'Business Days'])

In [ ]:
output_df.head(25)

In [ ]:
pricePt1Date

In [ ]:
dates_of_interest = pd.Series(pricePt1Date)
dates_of_interest

In [ ]:
dates_of_interest = dates_of_interest.apply(lambda x: (( datetime.datetime.strptime(x, '%Y-%m-%d') - datetime.timedelta(days=60)).strftime('%Y-%m-%d'))  )

In [ ]:
dates_of_interest

In [ ]:
pricePt2Date

In [ ]:
all_data_df = pd.DataFrame([])
for items in dates_of_interest.iteritems():
    date_1 = items[1]
    date_2 = pricePt2Date[items[0]]
    data_of_interest = yf.download(tickers = '^VIX SPY VXX', start = date_1, end = date_2, group_by = 'ticker', auto_adjust  = True)
    data_of_interest['Study_Item_Num'] = [(items[0]+1) for _ in range(len(data_of_interest))]
    all_data_df = all_data_df.append(data_of_interest);

In [ ]:
len(all_data_df.index)

In [ ]:
all_data_df.head(1000)

In [ ]:
output_dir = '../data/vix_analysis'
if not os.path.exists(output_dir):
    os.mkdir(output_dir);
    
output_file = 'data.xlsx';

file_path = os.path.join(output_dir, output_file)

all_data_df.to_excel(file_path)